In [1]:
import numpy as np
import scipy.linalg
import keras
import itertools
from hmmlearn import hmm

Using TensorFlow backend.


In [2]:
def whitening(z):
    x = z - np.mean(z, axis=0)
    c = np.linalg.inv(scipy.linalg.sqrtm(x.T @ x / len(x)))
    x = c @ x.T
    return x.T

In [3]:
def tae(hidden, latent_dim, dropout=0.):
    in_ = x = keras.layers.Input((3,))
    for units in hidden:
        x = keras.layers.Dense(units)(x)
        if dropout:
            x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.LeakyReLU(0.001)(x)
    x = keras.layers.Dense(latent_dim, name='latent')(x)
    for units in hidden[::-1]:
        x = keras.layers.Dense(units)(x)
        if dropout:
            x = keras.layers.Dropout(dropout)(x)
        x = keras.layers.LeakyReLU(0.001)(x)
    out_ = keras.layers.Dense(3)(x)
    return keras.models.Model(in_, out_)

In [4]:
def metastate_accuracy(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    metastates_true = np.unique(y_true)
    metastates_pred = np.unique(y_pred)
    assert len(metastates_true) == len(metastates_pred)
    accuracies = []
    for configuration in itertools.permutations(metastates_pred):
        correct = 0
        for metastate_true, metastate_pred in zip(metastates_true, configuration):
            correct += np.sum(y_pred[np.where(y_true == metastate_true)] == metastate_pred)
        accuracy = correct / len(y_true)
        accuracies.append(accuracy)
    return np.max(accuracies)

In [6]:
data = np.load('dimredux-challenge-01-data.npz')
data_x = data['data_x']
validation_x = data['validation_x']
validation_y = data['validation_y']

In [7]:
tau = 1
num_metastates = 4
x_train = whitening(data_x[:-tau])
y_train = whitening(data_x[tau:])

x_val = whitening(validation_x[:-tau])
y_val = whitening(validation_x[tau:])

model = tae(hidden=[100, 200], latent_dim=1, dropout=0.5)
model.compile(optimizer='adam', loss='mse')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)  

In [8]:
encoder = keras.models.Model(model.input, model.get_layer('latent').output)

In [9]:
model.fit(x_train, y_train, batch_size=256, epochs=10,
          verbose=2, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 99999 samples, validate on 999 samples
Epoch 1/10
 - 2s - loss: 0.7614 - val_loss: 0.6697
Epoch 2/10
 - 1s - loss: 0.6932 - val_loss: 0.6525
Epoch 3/10
 - 1s - loss: 0.6819 - val_loss: 0.6469
Epoch 4/10
 - 2s - loss: 0.6781 - val_loss: 0.6445
Epoch 5/10
 - 2s - loss: 0.6759 - val_loss: 0.6472
Epoch 6/10
 - 1s - loss: 0.6741 - val_loss: 0.6457
Epoch 7/10
 - 1s - loss: 0.6742 - val_loss: 0.6461
Epoch 8/10
 - 1s - loss: 0.6737 - val_loss: 0.6448
Epoch 9/10
 - 1s - loss: 0.6732 - val_loss: 0.6436
Epoch 10/10
 - 1s - loss: 0.6729 - val_loss: 0.6449


In [10]:
x_val = whitening(validation_x)
x_val_encoded = encoder.predict(x_val)

metastate_model = hmm.GaussianHMM(n_components=num_metastates, covariance_type='full', n_iter=100)
metastate_model.fit(x_val_encoded)

validation_y_pred = metastate_model.predict(x_val_encoded)

val_acc = metastate_accuracy(validation_y, validation_y_pred)

In [11]:
print('validation accuracy: {:.2%}'.format(val_acc))

validation accuracy: 100.00%


In [16]:
data_x = whitening(data_x)
data_x_encoded = encoder.predict(data_x)
data_y = metastate_model.predict(data_x_encoded)

In [18]:
assert(data_y.shape == (100000,))
np.save('prediction.npy', data_y)